In [1]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import yfinance as yf
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from langchain.chains import LLMChain
import json

In [2]:
import yfinance as yf
import json

@tool
def get_stock_data(ticker):
    """This function gets a JSON of information that contains price history,
    income statement, balance sheet, and cashflow of the ticker company from Yahoo Finance"""
    # ticker 가져오기
    stock = yf.Ticker(ticker)
        
    # 필요한 정보 가져오기
    hist = stock.history(period="1y")
    income_stmt = stock.financials
    balance_sheet = stock.balance_sheet
    cashflow = stock.cashflow
        
    # 데이터를 불러오는걸 실패했을 때
    if hist.empty:
        return json.dumps({"error": f"No data found for {ticker}"})
        
    # 불러온 정보들을 json으로 묶는다.
    financials = {
        "hist": json.loads(hist.to_json(date_format="iso")),
        "income_stmt": json.loads(income_stmt.to_json(date_format="iso")),
        "balance_sheet": json.loads(balance_sheet.to_json(date_format="iso")),
        "cashflow": json.loads(cashflow.to_json(date_format="iso"))
    }
        
    #json을 문자열로 변환한다.
    financials_str = json.dumps(financials)
        
    return financials_str


In [3]:
#llm 불러오기
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

In [4]:
# 도구 바인딩
tools = [get_stock_data]
llm_with_tools = llm.bind_tools(tools)

In [5]:
# 프롬프트 템플릿 정의
my_prompt_template = PromptTemplate.from_template(
    """
    You are an analyst for me.
    Search financial data for {company_name} and
    answer whether I should buy the stock or not, and explain the reason briefly.
    """
)

In [6]:
my_prompt_template

PromptTemplate(input_variables=['company_name'], template='\n    You are an analyst for me.\n    Search financial data for {company_name} and\n    answer whether I should buy the stock or not, and explain the reason briefly.\n    ')

In [7]:
user_template=my_prompt_template.format(company_name='Microsoft')
user_template

'\n    You are an analyst for me.\n    Search financial data for Microsoft and\n    answer whether I should buy the stock or not, and explain the reason briefly.\n    '

In [8]:
ai_message=llm_with_tools.invoke(user_template)

In [9]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AUT4WtJiIVUg83wPLrYpUvJg', 'function': {'arguments': '{"ticker": "MSFT"}', 'name': 'get_stock_data'}, 'type': 'function'}]})

In [10]:
ai_message.additional_kwargs

{'tool_calls': [{'id': 'call_AUT4WtJiIVUg83wPLrYpUvJg',
   'function': {'arguments': '{"ticker": "MSFT"}', 'name': 'get_stock_data'},
   'type': 'function'}]}

In [11]:
total_message=[HumanMessage(user_template)]

In [12]:
total_message.append(ai_message)

In [13]:
total_message

[HumanMessage(content='\n    You are an analyst for me.\n    Search financial data for Microsoft and\n    answer whether I should buy the stock or not, and explain the reason briefly.\n    '),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AUT4WtJiIVUg83wPLrYpUvJg', 'function': {'arguments': '{"ticker": "MSFT"}', 'name': 'get_stock_data'}, 'type': 'function'}]})]

In [14]:
for tool_call in ai_message.additional_kwargs['tool_calls']:
    selected_tool = {"get_stock_data": get_stock_data}[tool_call['function']['name'].lower()]
    arguments = tool_call['function']['arguments']
    arguments_dict = json.loads(arguments)
    tool_output = selected_tool.invoke(arguments_dict)
    total_message.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [17]:
total_message

[HumanMessage(content='\n    You are an analyst for me.\n    Search financial data for Microsoft and\n    answer whether I should buy the stock or not, and explain the reason briefly.\n    '),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AUT4WtJiIVUg83wPLrYpUvJg', 'function': {'arguments': '{"ticker": "MSFT"}', 'name': 'get_stock_data'}, 'type': 'function'}]}),
 ToolMessage(content='{"hist": {"Open": {"2023-07-20T04:00:00.000Z": 350.8256505507, "2023-07-21T04:00:00.000Z": 346.4399304532, "2023-07-24T04:00:00.000Z": 343.1655511952, "2023-07-25T04:00:00.000Z": 344.4157202621, "2023-07-26T04:00:00.000Z": 338.7897507204, "2023-07-27T04:00:00.000Z": 337.8372067165, "2023-07-28T04:00:00.000Z": 331.0800604487, "2023-07-31T04:00:00.000Z": 334.3048601292, "2023-08-01T04:00:00.000Z": 332.5882719068, "2023-08-02T04:00:00.000Z": 331.0404016843, "2023-08-03T04:00:00.000Z": 323.4695700063, "2023-08-04T04:00:00.000Z": 329.3039607696, "2023-08-07T04:00:00.000Z": 325.8212144004

In [15]:
message=llm_with_tools.invoke(total_message)

In [16]:
print(message.content)

### Should You Buy Microsoft Stock?

Based on the financial data provided for Microsoft (ticker: MSFT), here are key points to consider:

1. **Revenue and Earnings Growth**:
    - **Revenue**: Microsoft's revenue has been steadily increasing. For the fiscal year ending June 30, 2023, total revenue was $211.92 billion, up from $198.27 billion in 2022, and $168.08 billion in 2021.
    - **Net Income**: Net income also shows a positive trend, with $72.36 billion in 2023 compared to $72.74 billion in 2022, and $61.27 billion in 2021.

2. **Profitability**:
    - **Gross Profit**: The gross profit margin for 2023 was about 68.9% ($146.05 billion gross profit on $211.92 billion revenue), indicating strong profitability.
    - **Operating Income**: Operating income for 2023 was $88.52 billion, showing a substantial increase from previous years ($83.38 billion in 2022 and $69.92 billion in 2021).
    - **EBITDA**: The EBITDA for 2023 was $105.14 billion, indicating robust earnings before inter